<a href="https://colab.research.google.com/github/mgmontealegre/03MAIR--Algoritmos-de-Optimizacion--2020/blob/master/Mario_Garcia_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Actividad Guiada 2 de Algoritmos de Optimización

Mario García

Importamos las librerías necesarias

In [ ]:
import numpy as np
import random
import itertools
from functools import wraps
import math
import matplotlib.pyplot as plt
from scipy.optimize import approx_fprime as gradiente

Viaje por el río

In [ ]:
tarifas=[[0,5,4,3,999,999,999],
        [999,0,999,2,3,999,1],
        [999,999,0,1,999,4,10],
        [999,999,999,0,5,6,9],
        [999,999,999,999,0,999,4],
        [999,999,999,999,999,0,3],
        [999,999,999,999,999,999,0]]

def precio (tarifas):
  n = len(tarifas[0])
  precio = [[9999]*n for i in [9999]*n ]
  path = [[""]*n for i in [""]*n]
  
  for i in range(n-1):
      for j in range(i+1,n):
        minimum = tarifas[i][j]
        path[i][j] = i
        
        for k in range (i,j):
          if precio[i][k] + tarifas[k][j] < minimum:
            minimum = precio[i][k] + tarifas[k][j]
            path[i][j] = k
        precio[i][j] = minimum
  return precio, path

precio,path= precio(tarifas)
print(precio)
print(path)

def calcular_ruta(path, origen, destino):
  if origen==destino:
    return origen
  else:
    return str(calcular_ruta(path, origen, path[origen][destino])) + '-' + str(path[origen][destino])
  
calcular_ruta(path,1,5)

[[9999, 5, 4, 3, 8, 8, 6], [9999, 9999, 999, 2, 3, 8, 1], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, 999, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]
[['', 0, 0, 0, 1, 2, 1], ['', '', 1, 1, 1, 3, 1], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]


'1-1-3'

Búsqueda en grafos, ramificación y poda (asignación de tareas)

In [ ]:
def calcular_tiempo(f):
  @wraps(f)
  def cronometro(*args, **kwargs):
    t_inicial = time()
    salida= f(*args,**kwargs)
    t_final= time()
    print('Tiempo transcurrido ( en segundos): {}', format(t_final - t_inicial))
    return salida
  return cronometro

COSTES=[[11,12,18,40],
       [14,15,13,22],
       [11,17,19,23],
       [17,14,20,28]]

def valor(S,COSTES):
    VALOR=0
    for i in range(len(S)):
      VALOR+= COSTES[S[i]][i]
      
    return VALOR
  
valor ((0,1,2,3), COSTES)


def fuerza_bruta(COSTES):
    mejor_valor=10e10
    mejor_solucion= ''
    
    for S in list(itertools.permutations(range(len(COSTES)))):
      valor_tmp = valor(S, COSTES)
      if valor_tmp < mejor_valor:
        mejor_solucion=S
        mejor_valor=valor_tmp
        
    print("La mejor solucion es :" ,mejor_solucion)
    
fuerza_bruta(COSTES)


def CI(S,COSTES):
  VALOR=0
  #Valores Establecidos
  for i in range(len(S)):
    VALOR+= COSTES[i][S[i]]
    
 #Estimacion
  for i in range( len(COSTES)):
    if i not in S:
      VALOR+= min([COSTES[j][i] for j in range(len(S), len(COSTES))])
  return VALOR    


CI((2,0), COSTES)

def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS:append({'S': NODO+(i,)})
  return HIJOS


crear_hijos((0,2), 4)

def ramificacion_y_poda(COSTES):
  DIMENSION= len(COSTES)
  #(0,1,2,3)
  MEJOR_SOLUCION= tuple(i for i in range(DIMENSION))
  
  CotaSup= valor(MEJOR_SOLUCION,COSTES)
  
  NODOS=[]
  NODOS.append({'S':(), 'ci': CI((), COSTES) })
  
 
  
  iteracion=0
  
  while (len(NODOS)>0):
    iteracion+=1
    print('\n#', iteracion)
    
    nodo_prometedor = min(NODOS, key=lambda x:x['ci'])
    
    #Hijos
    
    HIJOS= [{'S':x['S'], 'ci': CI(x['S'], COSTES)} for x in crear_hijos(nodo_prometedor['S'], DIMENSION)]
    
    NODO_FINAL = [x for x in HIJOS if len(x['S']) ==DIMENSION]
    
    if len(NODO_FINAL)>0:
      if NODO_FINAL[0]['ci']< CotaSup:
        CotaSup=NODO_FINAL[0]['ci']
        MEJOR_SOLUCION=NODO_FINAL[0]
        
    #poda    
    HIJOS=[x for x in HIJOS if x['ci']<CotaSup]
    
    #Eliminamos el nodo expandido
    NODOS= [ x for x in NODOS if x['S'] != nodo_prometedor['S']]
    #Añadimos los hijos
    NODOS.extend(HIJOS)
    
    print(NODOS)
    
  print('La mejor solucion es:', MEJOR_SOLUCION)
  
ramificacion_y_poda(COSTES)

La mejor solucion es : (0, 3, 1, 2)

# 1
[]
La mejor solucion es: (0, 1, 2, 3)


Descenso gradiente

In [ ]:
# Generamos la función
f = lambda X: np.sin(1/2 * X[0]**2 - 1/4 * X[1]**2 + 3) * np.cos(2 * X[0] + 1 - np.e**X[1])
 
res, rango = 100, 4
X = np.linspace(-rango, rango, res)
Y = np.linspace(-rango, rango, res)
Z = np.zeros((res, res))
 
for ix,x in enumerate(X):
  for iy,y in enumerate(Y):
    Z[iy,ix] = f([x,y])
 
plt.contourf(X, Y, Z, res)
plt.colorbar()
 
P = [ np.random.uniform(-3,3), np.random.uniform(-3,3) ]
plt.plot(P[0], P[1], 'o', c='white')
 
# Tasa de aprendizaje
TA = 0.005 
 
for _ in range(500):
  grad = gradiente(P, f, epsilon=1e-6)
  P[0], P[1] = P[0] - TA*grad[0], P[1]-TA*grad[1]
  plt.plot(P[0], P[1], 'o', c='red')
  
plt.plot(P[0], P[1], 'o', c='green')
plt.show()
print('Solucion:', P, f(P))